# ToDo:
- preprocess text - lemmatization, remove stop words etc. 
- utlise preprocessed text
- Try DistilBert https://www.kaggle.com/code/alexia/kerasnlp-starter-notebook-disaster-tweets
- sklearn pipelines
- cross validation


In [19]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn import feature_extraction, linear_model, model_selection, preprocessing
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.metrics import f1_score, r2_score, accuracy_score
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
import xgboost
import spacy

In [20]:
nlp = spacy.load('en_core_web_sm')

In [21]:
train_df = pd.read_csv("/kaggle/input/nlp-getting-started/train.csv")
test_df = pd.read_csv("/kaggle/input/nlp-getting-started/test.csv")

## Preproc

In [22]:
def lemmatize_remove_stop(doc):
    return (" ".join([token.lemma_ for token in nlp(doc) if not token.is_stop])).strip().lower()

In [23]:
train_df['text_preproc'] = train_df['text'].apply(lemmatize_remove_stop)
test_df['text_preproc'] = test_df['text'].apply(lemmatize_remove_stop)

In [24]:
X = train_df.set_index('id')
y = train_df.set_index('id')['target']

X_test = test_df.set_index('id')

In [25]:
count_vectorizer = feature_extraction.text.CountVectorizer(max_features=5000)
X_bow_raw = count_vectorizer.fit_transform(X['text'])
X_test_bow_raw = count_vectorizer.transform(X_test['text'])

count_vectorizer2 = feature_extraction.text.CountVectorizer(max_features=5000)
X_bow = count_vectorizer2.fit_transform(X['text_preproc']);
X_test_bow = count_vectorizer2.transform(X_test['text_preproc']);

In [26]:
tfidf_vectorizer = feature_extraction.text.TfidfVectorizer(max_features=5000)
X_tfidf_raw = tfidf_vectorizer.fit_transform(X['text'])
X_test_tfidf_raw = tfidf_vectorizer.transform(X_test['text'])

tfidf_vectorizer2 = feature_extraction.text.TfidfVectorizer(max_features=5000)
X_tfidf = tfidf_vectorizer2.fit_transform(X['text_preproc']);
X_test_tfidf = tfidf_vectorizer2.transform(X_test['text_preproc']);

In [27]:
# X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.33, random_state=42)

In [28]:
# # Bag of words

# count_vectorizer = feature_extraction.text.CountVectorizer(max_features=5000)
# X_bow = count_vectorizer.fit_transform(X['text']);

# X_train_bow = count_vectorizer.transform(X_train['text'])
# X_test_bow = count_vectorizer.transform(X_test['text'])

# # TF-IDF

# tf_idf_trans = feature_extraction.text.TfidfVectorizer(max_features=5000)
# X_tf_idf = tf_idf_trans.fit_transform(X['text'])

# X_train_tf_idf = tf_idf_trans.transform(X_train)
# X_test_tf_idf = tf_idf_trans.transform(X_test)

## Modelling

In [29]:
def cross_validate_helper(model, X, y):
    return pd.DataFrame(cross_validate(model, X, y, scoring=["f1", "accuracy", "r2"], return_train_score=True)).agg(['mean', 'std'])

In [30]:
experiments = {"naivebayes_tfidf_preproc":dict(model=MultinomialNB(), X=X_tfidf, y=y),
               "naivebayes_bow_preproc":dict(model=MultinomialNB(), X=X_bow, y=y),
                "naivebayes_tfidf_raw":dict(model=MultinomialNB(), X=X_tfidf_raw, y=y),
                "naivebayes_bow_raw":dict(model=MultinomialNB(), X=X_bow_raw, y=y),    
}

In [31]:
pd.DataFrame([cross_validate_helper(**experiment_args).unstack().rename(experiment_name) 
              for experiment_name, experiment_args in experiments.items()]
            ).sort_values(('test_f1','mean'),ascending=False)

fit_time           score_time             test_f1  \
                              mean       std       mean       std      mean   
naivebayes_bow_raw        0.004820  0.000103   0.004365  0.000121  0.676385   
naivebayes_tfidf_raw      0.005429  0.000345   0.004516  0.000126  0.661277   
naivebayes_bow_preproc    0.004613  0.000149   0.004268  0.000097  0.656242   
naivebayes_tfidf_preproc  0.004930  0.000267   0.004450  0.000306  0.643062   

                                    train_f1           test_accuracy  \
                               std      mean       std          mean   
naivebayes_bow_raw        0.044076  0.825405  0.005619      0.722453   
naivebayes_tfidf_raw      0.058977  0.815621  0.006054      0.736509   
naivebayes_bow_preproc    0.050613  0.843953  0.005498      0.697237   
naivebayes_tfidf_preproc  0.062914  0.837367  0.005804      0.712604   

                                   train_accuracy             test_r2  \
                               std           mean       std      mean   
naivebayes_bow_raw        0.029429       0.859024  0.004497 -0.132609   
naivebayes_tfidf_raw      0.033414       0.857908  0.004023 -0.075245   
naivebayes_bow_preproc    0.039899       0.872488  0.003943 -0.235515   
naivebayes_tfidf_preproc  0.038916       0.871470  0.003957 -0.172799   

                                    train_r2            
                               std      mean       std  
naivebayes_bow_raw        0.120177  0.424711  0.018366  
naivebayes_tfidf_raw      0.136380  0.420154  0.016426  
naivebayes_bow_preproc    0.162934  0.479653  0.016100  
naivebayes_tfidf_preproc  0.158850  0.475499  0.016157

In [32]:
# def fit_return_metrics(X_train_input, y_train_input, X_test_input,y_test_input, model):
#     """Fit the model, and return train and test metrics"""

#     model.fit(X_train_input, y_train_input);
#     y_train_pred = model.predict(X_train_input)
#     y_test_pred = model.predict(X_test_input)

#     # train
    
#     metric_set = dict(f1=f1_score, 
#                       r2=r2_score,
#                       accuracy = accuracy_score)
    
#     def metrics_dict(y_act, y_pred):
#         return{metric : metric_func(y_act, y_pred) for metric, metric_func in metric_set.items()}
      
#     metrics_list = [dict(regime="train", **metrics_dict(y_train_input,y_train_pred)),
#                    dict(regime="test", **metrics_dict(y_test_input, y_test_pred))
#                    ]
    
#     return pd.DataFrame(metrics_list).set_index("regime").unstack()

In [33]:
# pd.DataFrame([fit_return_metrics(X_train_bow, y_train, X_test_bow,y_test,xgboost.XGBClassifier()).rename('xgboost_bow'),
#               fit_return_metrics(X_train_tf_idf, y_train, X_test_tf_idf,y_test,xgboost.XGBClassifier()).rename('xgboost_tfidf'),
# #               fit_return_metrics(X_train_bow.toarray(), y_train, X_test_bow.toarray(),y_test,GaussianProcessClassifier()).rename('gaussianprocess_bow'),
# #               fit_return_metrics(X_train_bow.toarray(), y_train, X_test_tf_idf.toarray(),y_test,GaussianProcessClassifier()).rename('gaussianprocess_tfidf'),
# #               fit_return_metrics(X_train_bow.toarray(), y_train, X_test_bow.toarray(),y_test,MultinomialNB()).rename('naivebayes_bow'),
# #               fit_return_metrics(X_train_bow.toarray(), y_train, X_test_tf_idf.toarray(),y_test,MultinomialNB()).rename('naivebayes_tfidf')
#              ])
 
 

In [34]:
# see this notebook for BERT implementation 
# https://www.kaggle.com/code/alexia/kerasnlp-starter-notebook-disaster-tweets

# Bert

See these: 
- https://www.kaggle.com/code/pritishmishra/text-classification-with-distilbert-92-accuracy
- https://huggingface.co/docs/transformers/tasks/sequence_classification

Going with Pytorch as the API seems way easier

In [39]:
from transformers import AutoTokenizer
import torch
import datasets

tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [41]:
X_dataset = datasets.Dataset.from_pandas(X[['text', 'target']].rename(columns={'target':'label'})).train_test_split(test_size=0.2)
X_dataset_tokenized = X_dataset.map(preprocess_function, batched=True)

  0%|          | 0/7 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [42]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)#, return_tensors="tf")

In [70]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

id2label = {0: 0, 1:1}
label2id = {0: 0, 1: 1}

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id)

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_projector.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'pre_classifier.

In [71]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}

In [72]:
training_args = TrainingArguments(
    output_dir="natural_disaster_tweets_model",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    push_to_hub=False,
    # added these
    metrics_for_best_model='f1',
    greater_is_better=True,
    eval_steps=100
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=X_dataset_tokenized["train"],
    eval_dataset=X_dataset_tokenized["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.409205,0.831254,0.830331
2,0.410900,0.427987,0.840446,0.839787


TrainOutput(global_step=762, training_loss=0.38080195429444, metrics={'train_runtime': 2102.1196, 'train_samples_per_second': 5.794, 'train_steps_per_second': 0.362, 'total_flos': 171458866979928.0, 'train_loss': 0.38080195429444, 'epoch': 2.0})

In [86]:
preds_output = trainer.predict(X_dataset_tokenized["test"])

In [87]:
preds_output.metrics

{'test_loss': 0.4092045724391937,
 'test_accuracy': 0.8312541037426132,
 'test_f1': 0.830330939313283,
 'test_runtime': 69.9307,
 'test_samples_per_second': 21.779,
 'test_steps_per_second': 1.373}

In [88]:
y_preds = np.argmax(preds_output.predictions, axis=1)

## **below is scratchy code**

In [ ]:
from transformers import create_optimizer
import tensorflow as tf

batch_size = 16
num_epochs = 5
batches_per_epoch = len(X_dataset_tokenized) // batch_size
total_train_steps = int(batches_per_epoch * num_epochs)
optimizer, schedule = create_optimizer(init_lr=2e-5, num_warmup_steps=0, num_train_steps=total_train_steps)

In [ ]:
from datasets import Dataset

tf_train_set = model.prepare_tf_dataset(
    X_dataset_tokenized['train'],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator,
)

tf_validation_set = model.prepare_tf_dataset(
    X_dataset_tokenized['test'],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator,
)

In [ ]:
model.compile(optimizer=optimizer) 

In [ ]:
from transformers.keras_callbacks import KerasMetricCallback

metric_callback = KerasMetricCallback(metric_fn=compute_metrics, eval_dataset=tf_validation_set)

In [118]:
model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=metric_callback)

Epoch 1/3
380/380 [==============================] - ETA: 0s - loss: 0.4406

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 model.fit(x=tf_train_set, validation_data=tf_validation_set, epochs=3, callbacks=metric_     │
│   2                                                                                              │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/keras/utils/traceback_utils.py:70 in error_handler       │
│                                                                                                  │
│    67 │   │   │   filtered_tb = _process_traceback_frames(e.__traceback__)                       │
│    68 │   │   │   # To get the full stack trace, call:                                           │
│    69 │   │   │   # `tf.debugging.disable_traceback_filtering()`                                 │
│ ❱  70 │   │   │   raise e.with_traceback(filtered_tb) from None                                  │
│    71 │   │   finally:                                                                           │
│    72 │   │   │   del filtered_tb                                                                │
│    73                                                                                            │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/keras_callbacks.py:256 in on_epoch_end      │
│                                                                                                  │
│   253 │   │   all_preds = self._postprocess_predictions_or_labels(prediction_list)               │
│   254 │   │   all_labels = self._postprocess_predictions_or_labels(label_list)                   │
│   255 │   │                                                                                      │
│ ❱ 256 │   │   metric_output = self.metric_fn((all_preds, all_labels))                            │
│   257 │   │   if not isinstance(metric_output, dict):                                            │
│   258 │   │   │   raise TypeError(                                                               │
│   259 │   │   │   │   f"metric_fn should return a dict mapping metric names to values but inst   │
│                                                                                                  │
│ in compute_metrics:2                                                                             │
│                                                                                                  │
│   1 def compute_metrics(pred):                                                                   │
│ ❱ 2 │   labels = pred.label_ids                                                                  │
│   3 │   preds = pred.predictions.argmax(-1)                                                      │
│   4 │   f1 = f1_score(labels, preds, average="weighted")                                         │
│   5 │   acc = accuracy_score(labels, preds)                                                      │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'tuple' object has no attribute 'label_ids'

In [103]:
from transformers import Trainer, TrainingArguments

batch_size = 64
logging_steps = len(X_dataset_tokenized['train']) // batch_size
model_name = f"distilbert-finetuned-disastertweets"

training_args = TrainingArguments(
    output_dir=model_name,
    num_train_epochs=2,
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    disable_tqdm=False,
    logging_steps=logging_steps,
    log_level="error",
    report_to="none"
)

In [108]:
trainer = Trainer(
    model=model,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=X_dataset_tokenized['train'],
    eval_dataset=X_dataset_tokenized['test'],
    tokenizer=tokenizer,
)

trainer.train()

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱  1 trainer = Trainer(                                                                          │
│    2 │   model=model,                                                                            │
│    3 │   args=training_args,                                                                     │
│    4 │   compute_metrics=compute_metrics,                                                        │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:511 in __init__                  │
│                                                                                                  │
│    508 │   │   self.tokenizer = tokenizer                                                        │
│    509 │   │                                                                                     │
│    510 │   │   if self.place_model_on_device and not getattr(model, "is_loaded_in_8bit", False)  │
│ ❱  511 │   │   │   self._move_model_to_device(model, args.device)                                │
│    512 │   │                                                                                     │
│    513 │   │   # Force n_gpu to 1 to avoid DataParallel as MP will manage the GPUs               │
│    514 │   │   if self.is_model_parallel:                                                        │
│                                                                                                  │
│ /opt/conda/lib/python3.10/site-packages/transformers/trainer.py:754 in _move_model_to_device     │
│                                                                                                  │
│    751 │   │   self.callback_handler.remove_callback(callback)                                   │
│    752 │                                                                                         │
│    753 │   def _move_model_to_device(self, model, device):                                       │
│ ❱  754 │   │   model = model.to(device)                                                          │
│    755 │   │   # Moving a model to an XLA device disconnects the tied weights, so we have to re  │
│    756 │   │   if self.args.parallel_mode == ParallelMode.TPU and hasattr(model, "tie_weights")  │
│    757 │   │   │   model.tie_weights()                                                           │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'TFDistilBertForSequenceClassification' object has no attribute 'to'

# Submit best model

In [17]:
best_model = MultinomialNB()

best_model.fit(X_bow_raw, y)

y_pred_test  = best_model.predict(X_test_bow_raw)

In [18]:
# Create a submission dataframe
submission = pd.DataFrame({'id': test_df['id'], 'target': y_pred_test})

# Write the submission dataframe to a CSV file
submission.to_csv('submission.csv', index=False)

submission.head()

,id,target
0,0,1
1,2,1
2,3,1
3,9,1
4,11,1
